In [1]:
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Exercises III
# Use your get_db_url function to help you 
# explore the data from the chipotle database.
import pandas as pd
import numpy as np
from env import get_db_url # This allows you to find the get_db_url function from the env.py file. 
# from pydataset import data



In [2]:

# get the data from MySQL
# The below lines allow you to communicate with MySQL and retrive tables.
# This retrives the orders table from the Chipotle database
# If you need to change databases and tables just update "chipotle (database) and orders (table)"
url = get_db_url('chipotle') # Chipotle is the database were the orders table is coming from. Don't forget that Chipotle is a database not a table.
pd.read_sql(url) # This will allow you to see how many tables are in the database. 
orders_table_df = pd.read_sql('SELECT * FROM orders', url) # Orders is a table. The table is located in the mySQL
print('\n',orders_table_df)



         id  order_id  quantity                              item_name  \
0        1         1         1           Chips and Fresh Tomato Salsa   
1        2         1         1                                   Izze   
2        3         1         1                       Nantucket Nectar   
3        4         1         1  Chips and Tomatillo-Green Chili Salsa   
4        5         2         2                           Chicken Bowl   
...    ...       ...       ...                                    ...   
4617  4618      1833         1                          Steak Burrito   
4618  4619      1833         1                          Steak Burrito   
4619  4620      1834         1                     Chicken Salad Bowl   
4620  4621      1834         1                     Chicken Salad Bowl   
4621  4622      1834         1                     Chicken Salad Bowl   

                                     choice_description item_price  
0                                                   

In [3]:


# 1. What is the total price for each order?
print('\nExercises III, Question 1')

# ************** Option 1 **************
query = '''
SELECT order_id, SUM(CAST(REPLACE(item_price, '$', '') AS DECIMAL(10,2))) AS total_price
FROM orders
GROUP BY order_id
'''

total_price_df = pd.read_sql(query, url)
print(f'\n************** Option 1 **************\n',total_price_df)


# ************** Option 2 **************
# clean the item_price column
orders_table_df['item_price'] = orders_table_df['item_price']\
    .str.replace('$', '').astype(float)

# pivot the data to get the total price for each order
order_totals = pd.pivot_table(orders_table_df, values='item_price',\
                               index='order_id', aggfunc=np.sum)

print(f'\n************** Option 2 **************\n',order_totals,'\n')


# ************** Option 3 **************
order_price = orders_table_df.groupby('order_id').item_price.sum()





Exercises III, Question 1

************** Option 1 **************
       order_id  total_price
0            1        11.56
1            2        16.98
2            3        12.67
3            4        21.00
4            5        13.70
...        ...          ...
1829      1830        23.00
1830      1831        12.90
1831      1832        13.20
1832      1833        23.50
1833      1834        28.75

[1834 rows x 2 columns]

************** Option 2 **************
           item_price
order_id            
1              11.56
2              16.98
3              12.67
4              21.00
5              13.70
...              ...
1830           23.00
1831           12.90
1832           13.20
1833           23.50
1834           28.75

[1834 rows x 1 columns] 



/var/folders/67/bss6mpl54l73122199y9h3r40000gn/T/ipykernel_66131/408197782.py:17: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  orders_table_df['item_price'] = orders_table_df['item_price']\


In [21]:


# 2. What are the most popular 3 items?
print('\nExercises III, Question 2')
# ************** Option 1 **************
popular_three = orders_table_df.groupby('item_name')['quantity'].sum()
top_three = popular_three.sort_values(ascending=False).head(3)
print(f'\n************** Option 1 **************\n',top_three)


# ************** Option 2 **************
most_popular = pd.pivot_table(data=orders_table_df, index='item_name', values='quantity', aggfunc='sum')
three_popular = most_popular.nlargest(3, 'quantity')
print(f'\n************** Option 2 **************\n',three_popular) 

# ************** Option 3 **************
popular_three_items = orders_table_df.groupby('item_name').quantity.sum().sort_values(ascending=False).head()
print(f'\n************** Option 3 **************\n',popular_three_items)



Exercises III, Question 2

************** Option 1 **************
 item_name
Chicken Bowl           761
Chicken Burrito        591
Chips and Guacamole    506
Name: quantity, dtype: int64

************** Option 2 **************
                      quantity
item_name                    
Chicken Bowl              761
Chicken Burrito           591
Chips and Guacamole       506

************** Option 3 **************
 item_name
Chicken Bowl           761
Chicken Burrito        591
Chips and Guacamole    506
Steak Burrito          386
Canned Soft Drink      351
Name: quantity, dtype: int64


In [5]:


# 3. Which item has produced the most revenue?
print('\n\nExercises III, Question 3')

# ************** Option 1 **************

# Use the pivot_table() method to group orders by item_name and order_id
#  and sum the quantity and item_price columns for each group
order_totals = pd.pivot_table(orders_table_df, index='item_name',\
                               values=['quantity', 'item_price'], aggfunc=np.sum)


# Created another column named "total_price" by multipling the quantity and item price
order_totals['total_price'] = order_totals['quantity'] * order_totals['item_price']

# Use the nlargest() method to return the row with the highest total item_price
most_revenue = order_totals.nlargest(1, 'total_price')

print(f'The item with the most Revenue\n\n',most_revenue)

# ************** Option 2 **************

orders_table_df[orders_table_df.item_name == 'Chicken Bowl'].sort_values('quantity')

most_revenue_opt2 = orders_table_df.groupby('item_name').item_price.sum().nlargest(1)
print(f'The item with the most Revenue - Option 2\n\n',most_revenue_opt2)



Exercises III, Question 3
The item with the most Revenue

               item_price  quantity  total_price
item_name                                      
Chicken Bowl     7342.73       761   5587817.53


In [7]:

# 4. Join the employees and titles DataFrames together.
print('\nExercises III, Question 4')
url = get_db_url('employees') # Employees is the database were the orders table is coming from.
# Don't forget that Employee is a database not a table.
titles_table_df = pd.read_sql('SELECT * FROM titles', url) # Title is a table. The table is located in the mySQL and was limited to 1000 rows
employees_table_df = pd.read_sql('SELECT * FROM employees', url) # Employees is a table. The table is located in the mySQLL and was limited to 1000 rows
print('\n',titles_table_df.head())
print('\n',employees_table_df.head())

# format: pd.merge(df1,df2, how='', on='column_name') 
# The how is asking if you want to do it by 'inner', 'outer', 'left', or 'right'.
# The on= can only be used if both tables have the same name (emp_no) in the columns if they have
#  differen names use the left_on='emp_num' and right_on='employee_number'. All this is assuming that the 
# values inside the columns are the same. 
# the "how='emp_no'" doesn't have to be listed because is a default. 
et_tt_merge_df = pd.merge(titles_table_df, employees_table_df, how='inner', on='emp_no')
print('\n',et_tt_merge_df)



Exercises III, Question 4

    emp_no            title   from_date     to_date
0   10001  Senior Engineer  1986-06-26  9999-01-01
1   10002            Staff  1996-08-03  9999-01-01
2   10003  Senior Engineer  1995-12-03  9999-01-01
3   10004         Engineer  1986-12-01  1995-12-01
4   10004  Senior Engineer  1995-12-01  9999-01-01

    emp_no  birth_date first_name last_name gender   hire_date
0   10001  1953-09-02     Georgi   Facello      M  1986-06-26
1   10002  1964-06-02    Bezalel    Simmel      F  1985-11-21
2   10003  1959-12-03      Parto   Bamford      M  1986-08-28
3   10004  1954-05-01  Chirstian   Koblick      M  1986-12-01
4   10005  1955-01-21    Kyoichi  Maliniak      M  1989-09-12

         emp_no            title   from_date     to_date  birth_date  \
0        10001  Senior Engineer  1986-06-26  9999-01-01  1953-09-02   
1        10002            Staff  1996-08-03  9999-01-01  1964-06-02   
2        10003  Senior Engineer  1995-12-03  9999-01-01  1959-12-03   
3    

In [12]:
et_tt_merge_df

,emp_no,title,from_date,to_date,birth_date,first_name,last_name,gender,hire_date
0,10001,Senior Engineer,1986-06-26,9999-01-01,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,Staff,1996-08-03,9999-01-01,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,Senior Engineer,1995-12-03,9999-01-01,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,Engineer,1986-12-01,1995-12-01,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10004,Senior Engineer,1995-12-01,9999-01-01,1954-05-01,Chirstian,Koblick,M,1986-12-01
...,...,...,...,...,...,...,...,...,...
443303,499997,Engineer,1987-08-30,1992-08-29,1961-08-03,Berhard,Lenart,M,1986-04-21
443304,499997,Senior Engineer,1992-08-29,9999-01-01,1961-08-03,Berhard,Lenart,M,1986-04-21
443305,499998,Senior Staff,1998-12-27,9999-01-01,1956-09-05,Patricia,Breugel,M,1993-10-13
443306,499998,Staff,1993-12-27,1998-12-27,1956-09-05,Patricia,Breugel,M,1993-10-13


In [17]:
et_tt_merge_df.describe(include='all')

,emp_no,title,from_date,to_date,birth_date,first_name,last_name,gender,hire_date
count,443308.000000,443308,443308,443308,443308,443308,443308,443308,443308
unique,NaN,7,6393,5888,4750,1275,1637,2,5434
top,NaN,Engineer,1998-10-25,9999-01-01,1962-03-09,Shahab,Mandell,M,1985-05-11
freq,NaN,115003,132,240124,141,437,338,266084,215
mean,253075.034430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,161853.292613,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,10001.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,84855.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,249847.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,424891.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:

# 5. For each title, find the hire date of the employee
# that was hired most recently with that title.
print('\nExercises III, Question 5')

# Examples of sorting and groupby
    # df.sort_values(by='col1', ascending=False)
    # format: df.groupby('column_name').agg_function()
    # df.groupby('room').max()[['math','reading','english']]

# This will give you the olddest date. 
# Utilizied .max() as assending
et_tt_merge_df.groupby('title').hire_date.min() 

# This will give you the most current date. 
# Utilizied .max() as dessending
recently_hire = et_tt_merge_df.groupby('title').hire_date.max()

print(f'The employees that were hired most recently by title: \n\n',recently_hire)



Exercises III, Question 5
The employee that was hired most recently by title: 

 title
Assistant Engineer    1999-12-24
Engineer              2000-01-28
Manager               1992-02-05
Senior Engineer       2000-01-01
Senior Staff          2000-01-13
Staff                 2000-01-12
Technique Leader      1999-12-31
Name: hire_date, dtype: object


In [ ]:

# 6. Write the code necessary to create a cross tabulation
# of the number of titles by department. (Hint: this 
# will involve a combination of SQL code to pull the 
# necessary data and python/pandas code to perform the
# manipulations.)
print('\nExercises III, Question 6')
url = get_db_url('employees')

employees_table_df
emp_dept = pd.read_sql
query = '''
    select *
    from dept_emp
        join departments
            using(dept_no)
'''

et_tt_merge_df.head()
emp_dept.head()


et_tt_dt_df = pd.merge(et_tt_merge_df)

pd.crosstab(et_tt_dt_df.title, et_tt_dt_df.dept_name)